# Doubly-driven anharmonic oscillator

Program developed by Adrien Poindron.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.signal import correlate, hilbert
from math import ceil

import os
import sys
sys.path.append("../Functions")
from calcium_constants import *

import time
from datetime import datetime
%load_ext watermark
%watermark

Last updated: 2025-01-06T10:11:14.046202+01:00

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.15.0

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 23.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



In [2]:
from matplotlib.pyplot import cm
cm = plt.get_cmap('turbo')
plt.rcParams['figure.figsize'] = [11.7, 8.3]
plt.rcParams.update({'font.size': 16})

In [ ]:
class DDDAO_2:
    '''
        Class to represent a 1D Doubly-Driven and Damped Anharmonic Oscillator (DDDAO).
        Uses a Runge-Kutta 4th order algorithm.
        The drives are electric tickle and NW.
        This class is used for experiments where the drive frequency or phase is sweeped.
        It requires a lot of data to dissipate the transients.
        The idea is to save only a small part of the variables to limit the memory usage.
    '''
    def __init__(self,
                V_tkl = 0.022,V_nw = 5,V_piezo = 5,
                omega_t = None,
                phi_nw = 0,                
                B = None,
                cooling_rate = None, nl_damping_rate = None,
                n_dt = 2000,
                i_init = None,i_drive = None, i_freq_sweep = None,
                add_string=''):
        '''
        Initialise the physical parameters of the trapm coupling with nw, tickle and laser.
        Also initialise the parameters of the numerical simulation itself.
        '''
        self.add_string = add_string
        
        print('=== INIT SIMU ===')

        # trap parameters
        self.omega_z = 422500 *2*np.pi
        self.omega_z_2 = self.omega_z **2
        # self.omega_z = omega_z if type(omega_z) is (list or numpy.ndarray) else 422500*2*np.pi
        # self.omega_z_2 = self.omega_z **2
        self.B = B if B is not None else 1.24*1e18 *2*np.pi # Anharmonicity
        
        # NW coupling parametrers
        self.d_offset = 33e-6
        self.d_nw = 250e-6 + self.d_offset
        self.z_nw = 100e-6 * self.d_nw / (self.d_nw-self.d_offset)
        self.omega_nw = 422500*2*np.pi
        self.phi_nw = phi_nw *np.pi/180 # 0 * np.pi/180 # np.pi

        self.V_nw = V_nw # 1

        self.V_piezo = V_piezo # 5
        self.A_nw  = 184.41690653263915e-9*self.V_piezo
        self.eps   = Coul_factor*C_e*1.84*1e-15*self.V_nw

        # tickle parameters
        self.d_tkl = 3e-3 /2
        self.V_tkl = V_tkl # 0.022
        self.omega_t = omega_t[0] if type(omega_t) is list or type(omega_t) is np.ndarray else omega_t
        self.phi_t = np.pi*0

        # coupling parameters
        self.A1 = ( - self.eps/(self.d_nw**3) * (1 - 3*self.z_nw**2/self.d_nw**2) ) * self.A_nw
        self.kappa = 0.03 # self.kappa = 0.03451
        self.A2 = -C_e*self.kappa*self.V_tkl/self.d_tkl

        print(f'NW  : A1/m = {self.A1/m_Ca:.3e}')
        print(f'Tkl : A2/m = {self.A2/m_Ca:.3e}')

        # cool parameters
        # self.Gamma = 30e3
        self.P397 = 150e-6 # 70e-6
        self.w397 = 120e-6 # 50e-6
        self.lam_397 = 396.959120*1e-9
        self.laser_parameters()
        self.friction_parameters()

        print(f'Saturation = {self.sat0:.5e}')
        print(f'Detuning = {self.detuning/Gamma_SP:.5e} Gamma')

        print(f'Beta = {self.beta:.5e}') # no unit
        print(f'Gamma = {self.Gamma:.5e}')
        
        self.cooling_rate = cooling_rate if cooling_rate is not None else self.Gamma # * 30 # cooling_rate or self.Gamma * 30
        self.nl_damping_rate = nl_damping_rate if nl_damping_rate is not None else 5.28*1e-9
        
        print(f'B = {self.B:.5e}')
        print(f'cooling_rate = {self.cooling_rate:.5e}') # 2.86013e+04
        print(f'nl_damping_rate = {self.nl_damping_rate:.5e}')

        # heat parameters
        self.kheat = 0

        # numerical parameters
        self.n_dt = n_dt # 5000 * 1 # 5000*3  Resolution (number of time-steps for one omega_z period)
        self.dt = 2*np.pi/(self.n_dt*self.omega_z) # Time-step duration
        self.h = self.dt # RK4 time-step
        self.i_init       = i_init       if i_init is not None else 0
        self.i_drive      = i_drive      if i_drive is not None else 0
        self.i_freq_sweep = i_freq_sweep if i_freq_sweep is not None else 0
        self.n_init  = self.n_dt*self.i_init # Simu duration (in ubnits of omega_z periods)
        self.n_drive = self.n_dt*self.i_drive # Simu duration (in ubnits of omega_z periods)
        self.n_freq_sweep = self.n_dt*self.i_freq_sweep # Simu duration (in ubnits of omega_z periods)

        self.n_total = self.n_init + self.n_drive + self.n_freq_sweep*len(omega_t)

        self.i_save = 500

        # self.time = np.arange(0,self.dt*self.n_total,self.dt)
        self.time = np.linspace(0,self.dt*self.n_total,self.n_total)
        self.step_act = 0
        self.r_z = np.zeros(self.n_init +1)
        self.v_z = np.zeros(self.n_init +1)
        self.a_z = np.zeros(self.n_init +1)

        self.r_z[0] = 10e-6
        self.a_z[0] = self.a_trap(0, self.r_z[0])

        self.k_RK4 = np.zeros((4,2))

    def laser_parameters(self):
        '''
        Laser parameters. Doppler cooling with 397 only.
        '''
        self.k397 = 2*np.pi/self.lam_397
        self.f_397 = c_light/self.lam_397
        self.detuning = (self.f_397 - f_397_Wan)*2*np.pi
        self.intensity = 4*self.P397/(np.pi*self.w397**2)
        # self.satI = 2*hbar*np.pi**2*c_light/(3*self.lam_397**3)*Gamma_SP
        self.satI = hbar*(2*np.pi*self.f_397)**3/(12*np.pi*c_light**2)*Gamma_SP
        self.sat0 = self.intensity/self.satI

    def friction_parameters(self):
        '''
        Constants of the anharmonicities and friction.
        '''
        self.beta = -4*self.detuning/Gamma_SP * self.sat0 / (1+self.sat0+4*(self.detuning/Gamma_SP)**2)**2 # no unit
        self.Gamma = self.beta * hbar*self.k397**2/m_Ca # 

    # Expressions for the various forces
    def a_trap(self, el_tiempo, la_posicion):
        return self.omega_z_2*(la_posicion) + self.B*la_posicion**3

    def a_cool(self, la_velocidad):
        return self.cooling_rate*la_velocidad

    def a_tickle(self, el_tiempo):
        return self.A2/m_Ca*np.cos(self.omega_t * el_tiempo + self.phi_t)

    def a_nw(self, el_tiempo):
        return self.A1/m_Ca*np.cos(self.omega_nw * el_tiempo + self.phi_nw)

    def a_nl_damping(self, la_velocidad):
        '''
        Non-linear damping force, choosen as cubic velocity term.
        '''
        return self.nl_damping_rate*la_velocidad**3

    # Derivatives
    def derivs_init(self,el_tiempo,la_posicion,la_velocidad):
        self.dydx_temp  = la_velocidad

        a_3 = self.a_trap(el_tiempo, la_posicion)
        a_4 = self.a_cool(la_velocidad)
        a_5 = self.a_nl_damping(la_velocidad)
        self.dydx2_temp = - a_3 - a_4 - a_5

    def derivs(self,el_tiempo,la_posicion,la_velocidad):
        '''
        Compute the derivative for the RK4 algorithm.
        '''
        self.dydx_temp  = la_velocidad

        a_1 = self.a_tickle(el_tiempo)
        a_2 = self.a_nw(el_tiempo)
        a_3 = self.a_trap(el_tiempo, la_posicion)
        a_4 = self.a_cool(la_velocidad)
        a_5 = self.a_nl_damping(la_velocidad)
        self.dydx2_temp = a_1 + a_2 - a_3 - a_4 - a_5

    # The RK4 coefficients
    def update_rk4(self,f_derivs):
        '''
        Updates the RK4 factors, given the function for the derivative (derivs).
        '''
        f_derivs(self.time_act,self.r_z[self.step_act],self.v_z[self.step_act])
        self.k_RK4[0,:] = self.h*np.array([self.dydx_temp, self.dydx2_temp])

        f_derivs(self.time_act+self.h/2,
                self.r_z[self.step_act] + self.k_RK4[0,0]/2,
                self.v_z[self.step_act] + self.k_RK4[0,1]/2)
        self.k_RK4[1,:] = self.h*np.array([self.dydx_temp, self.dydx2_temp])

        f_derivs(self.time_act+self.h/2,
                self.r_z[self.step_act] + self.k_RK4[1,0]/2,
                self.v_z[self.step_act] + self.k_RK4[1,1]/2)
        self.k_RK4[2,:] = self.h*np.array([self.dydx_temp, self.dydx2_temp])

        f_derivs(self.time_act+self.h,
                self.r_z[self.step_act] + self.k_RK4[2,0],
                self.v_z[self.step_act] + self.k_RK4[2,1])
        self.k_RK4[3,:] = self.h*np.array([self.dydx_temp, self.dydx2_temp])

    # Update r, v and a
    def update_rva(self):
        '''
        Updates position (r_z), velocity (v_z) and acceleration (a_z) following the RK4 factors.
        '''
        self.r_z[self.step_act+1] = self.r_z[self.step_act] + (self.k_RK4[0,0] + 2*self.k_RK4[1,0] + 2*self.k_RK4[2,0] + self.k_RK4[3,0])/6
        self.v_z[self.step_act+1] = self.v_z[self.step_act] + (self.k_RK4[0,1] + 2*self.k_RK4[1,1] + 2*self.k_RK4[2,1] + self.k_RK4[3,1])/6 # + np.sqrt(self.kheat*3*kb/m_Ca*self.dt)
        self.a_z[self.step_act+1] = self.dydx2_temp

        self.step_act += 1

    # Run init with all forces
    def run_init(self):
        '''
        Runs initialisation step where only trap and laser cooling applies.
        '''
        print('> INIT')
        print('f_t',self.omega_t/2/np.pi)
        for i,j in enumerate(self.time[:self.n_init]):
            self.time_act = j
            self.update_rk4(self.derivs) # derivs_init
            self.update_rva()
        self.i_sweep = 0
        
        self.create_filenames()
        self.save_data_end(self.time[:self.n_init])
        self.save_metadata()

        aux_r_z = self.r_z[-1]
        aux_v_z = self.v_z[-1]
        aux_a_z = self.a_z[-1]

        self.r_z = np.zeros(self.n_freq_sweep +1)
        self.v_z = np.zeros(self.n_freq_sweep +1)
        self.a_z = np.zeros(self.n_freq_sweep +1)

        self.r_z[0] = aux_r_z
        self.v_z[0] = aux_v_z
        self.a_z[0] = aux_a_z

        self.step_act = 0

    # Run drive
    def run_drive(self):
        '''
        Runs the full model with trap and laser cooling, tickle and NW.
        '''
        print('> DRIVE')
        # print(self.time[self.n_init])
        for i,j in enumerate(self.time[self.n_init:self.n_total]):
            self.time_act = j
            self.update_rk4(self.derivs)
            self.update_rva()

        # self.save_data()

    # Similar to run drive but omega_t is changed after self.n_freq_sweep periods
    # This function saves the last data points
    def run_freq_sweep(self):
        '''
        Runs the full model with trap and laser cooling, tickle and NW.
        '''
        print('> FREQUENCY SWEEP')
        print(f'{omega_t[0]/2/np.pi:.3e} to {omega_t[-1]/2/np.pi:.3e} in {len(omega_t):02d} steps')
        for self.k_sweep,l in enumerate(omega_t):
            self.omega_t = l
            self.i_sweep += 1
            print('f_t',self.omega_t/2/np.pi)
            for i,j in enumerate(self.time[self.n_init + self.n_freq_sweep*(self.k_sweep) : self.n_init+self.n_freq_sweep*(self.k_sweep+1) ]):
                self.time_act = j
                self.update_rk4(self.derivs)
                self.update_rva()

            self.create_filenames()
            self.save_data_end(self.time[self.n_init + self.n_freq_sweep*(self.k_sweep) : self.n_init+self.n_freq_sweep*(self.k_sweep+1) ])
            self.save_metadata()

            aux_r_z = self.r_z[-1]
            aux_v_z = self.v_z[-1]
            aux_a_z = self.a_z[-1]

            self.r_z = np.zeros(self.n_freq_sweep +1)
            self.v_z = np.zeros(self.n_freq_sweep +1)
            self.a_z = np.zeros(self.n_freq_sweep +1)

            self.r_z[0] = aux_r_z
            self.v_z[0] = aux_v_z
            self.a_z[0] = aux_a_z

            self.step_act = 0
    
    def create_filenames(self):
        # self.savename = f'ion_Vtkl{self.V_tkl*1e3:06.3f}_VNW{self.V_nw:05.2f}_Vpiezo{self.V_piezo:05.2f}_PhiNW{self.phi_nw*180/np.pi:06.2f}deg{self.add_string}'
        self.savename = f'ion_Vtkl{self.V_tkl*1e3:06.3f}_VNW{self.V_nw:05.2f}_Vpiezo{self.V_piezo:05.2f}_PhiNW{self.phi_nw*180/np.pi:06.2f}deg{self.add_string+self.i_sweep:04d}'
        self.savedir_date = f'{datetime.today().strftime("%y%m%d")}'
        self.savedir = f'/Users/adrien/Documents/SIMU/{self.savedir_date}/'
        
        print(self.savename)
        print(self.savedir)

        try:
            os.mkdir(self.savedir)
        except FileExistsError:
            pass

    def save_data(self):
        print('> SAVE DATA')
        filename = f'{self.savedir}rva_{self.savename}'
        np.savez(filename,
                time=self.time,
                r_z=self.r_z,
                v_z=self.v_z,
                a_z=self.a_z)

    def save_data_end(self,die_Zeit):
        print('> SAVE DATA END')
        filename = f'{self.savedir}rva_{self.savename}'
        np.savez(filename,
                time = die_Zeit[-self.i_save*self.n_dt:],
                r_z  = self.r_z[-self.i_save*self.n_dt:],
                v_z  = self.v_z[-self.i_save*self.n_dt:],
                a_z  = self.a_z[-self.i_save*self.n_dt:])

    def save_metadata(self):
        print('> SAVE METADATA')
        filename = f'{self.savedir}potentials_{self.savename}.txt'
        with open(filename,'w') as f:
            f.write(f'{self.omega_z}\n')
            f.write(f'{self.B}\n')
            f.write(f'{self.d_nw}\n')
            f.write(f'{self.z_nw}\n')
            f.write(f'{self.omega_nw}\n')
            f.write(f'{self.phi_nw}\n')
            f.write(f'{self.V_nw}\n')
            f.write(f'{self.V_piezo}\n')
            f.write(f'{self.V_tkl}\n')
            f.write(f'{self.omega_t}\n')
            f.write(f'{self.phi_t}\n')
            f.write(f'{self.kappa}\n')

        filename = f'{self.savedir}numericals_{self.savename}.txt'
        with open(filename,'w') as f:
            f.write(f'{self.n_dt}\n')
            f.write(f'{self.dt}\n')
            f.write(f'{self.h}\n')
            f.write(f'{self.n_init:d}\n')
            f.write(f'{self.n_drive:d}\n')
            f.write(f'{self.n_freq_sweep:d}\n')
            f.write(f'{self.n_total:d}\n')

        filename = f'{self.savedir}cooling_{self.savename}.txt'
        with open(filename,'w') as f:
            f.write(f'{self.P397}\n')
            f.write(f'{self.w397}\n')
            f.write(f'{self.lam_397}\n')
            f.write(f'{self.kheat}\n')
            f.write(f'{self.sat0}\n')
            f.write(f'{self.detuning}\n')
            f.write(f'{self.beta}\n')
            f.write(f'{self.Gamma}\n')
            f.write(f'{self.cooling_rate}\n')
            f.write(f'{self.nl_damping_rate}\n')

    def plot_position(self):
        fig, ax = plt.subplots(num='hola',clear=True)

        ax.plot(self.time*1e3,self.r_z*1e6)
        ax.set_xlabel('time (ms)')
        ax.set_ylabel('z (um)')
        ax.grid()
        
        plt.tight_layout()

    def plot_velocity(self):
        fig, ax = plt.subplots(num='hola',clear=True)

        ax.plot(self.time*1e3,self.v_z)
        ax.set_xlabel('time (ms)')
        ax.set_ylabel('v_z (m/s)')
        ax.grid()

        plt.tight_layout()

In [44]:
###############
## 25/01/07

# Execute frequency sweep
# with partial save of end of each frequency

print('.-=[*\/*\/*\/*\/*]=-.')
# print(f'Rel. phase : {float(j):06.2f}')
delta_omega_t = 6
omega_t = np.linspace(422-delta_omega_t,422+delta_omega_t,7)*1e3*2*np.pi

simulation = DDDAO_2(V_tkl = 0.00020768959957006468*5, V_nw = 0, V_piezo = 0,
                    omega_t = omega_t,
                    phi_nw = 0,
                    B = 1.24*1e18 *4*np.pi**2,
                    cooling_rate = 2*28 *2*np.pi, nl_damping_rate = 0.03566372451203145,
                    n_dt = 1000,
                    i_init = 2000, i_freq_sweep = 20000,
                    add_string=200) # f'_{500+i:04d}'

start = time.time()
simulation.run_init()
simulation.run_freq_sweep()
end = time.time()
print(f'Simulation duration : {end-start:.3f} s')
print()

.-=[*\/*\/*\/*\/*]=-.
=== INIT SIMU ===
NW  : A1/m = -0.000e+00
Tkl : A2/m = -5.000e+04
Saturation = 2.94267e+01
Detuning = -1.24444e+01 Gamma
Beta = 3.46824e-03
Gamma = 1.37689e+03
B = 4.89532e+19
cooling_rate = 3.51858e+02
nl_damping_rate = 3.56637e-02
> INIT
f_t 423500.0
ion_Vtkl01.038_VNW00.00_Vpiezo00.00_PhiNW000.00deg0200
/Users/adrien/Documents/SIMU/250107/
> SAVE DATA END
> SAVE METADATA
> FREQUENCY SWEEP
4.235e+05 to 4.275e+05 in 09 steps
f_t 423500.0
ion_Vtkl01.038_VNW00.00_Vpiezo00.00_PhiNW000.00deg0201
/Users/adrien/Documents/SIMU/250107/
> SAVE DATA END
> SAVE METADATA
f_t 424000.0
ion_Vtkl01.038_VNW00.00_Vpiezo00.00_PhiNW000.00deg0202
/Users/adrien/Documents/SIMU/250107/
> SAVE DATA END
> SAVE METADATA
f_t 424500.0
ion_Vtkl01.038_VNW00.00_Vpiezo00.00_PhiNW000.00deg0203
/Users/adrien/Documents/SIMU/250107/
> SAVE DATA END
> SAVE METADATA
f_t 425000.0
ion_Vtkl01.038_VNW00.00_Vpiezo00.00_PhiNW000.00deg0204
/Users/adrien/Documents/SIMU/250107/
> SAVE DATA END
> SAVE METADATA


In [45]:
###############
## 25/01/07

# Try frequency sweep
# with partial save of end of each frequency

print('.-=[*\/*\/*\/*\/*]=-.')
# print(f'Rel. phase : {float(j):06.2f}')
delta_omega_t = -6
omega_t = np.linspace(422-delta_omega_t,422+delta_omega_t,7)*1e3*2*np.pi

simulation = DDDAO_2(V_tkl = 0.00020768959957006468*5, V_nw = 0, V_piezo = 0,
                    omega_t = omega_t,
                    phi_nw = 0,
                    B = 1.24*1e18 *4*np.pi**2,
                    cooling_rate = 2*28 *2*np.pi, nl_damping_rate = 0.03566372451203145,
                    n_dt = 1000,
                    i_init = 2000, i_freq_sweep = 20000,
                    add_string=300) # f'_{500+i:04d}'

start = time.time()
simulation.run_init()
simulation.run_freq_sweep()
end = time.time()
print(f'Simulation duration : {end-start:.3f} s')
print()

.-=[*\/*\/*\/*\/*]=-.
=== INIT SIMU ===
NW  : A1/m = -0.000e+00
Tkl : A2/m = -5.000e+04
Saturation = 2.94267e+01
Detuning = -1.24444e+01 Gamma
Beta = 3.46824e-03
Gamma = 1.37689e+03
B = 4.89532e+19
cooling_rate = 3.51858e+02
nl_damping_rate = 3.56637e-02


> INIT
f_t 427500.0
ion_Vtkl01.038_VNW00.00_Vpiezo00.00_PhiNW000.00deg0300
/Users/adrien/Documents/SIMU/250107/
> SAVE DATA END
> SAVE METADATA
> FREQUENCY SWEEP
4.275e+05 to 4.235e+05 in 09 steps
f_t 427500.0
ion_Vtkl01.038_VNW00.00_Vpiezo00.00_PhiNW000.00deg0301
/Users/adrien/Documents/SIMU/250107/
> SAVE DATA END
> SAVE METADATA
f_t 427000.0
ion_Vtkl01.038_VNW00.00_Vpiezo00.00_PhiNW000.00deg0302
/Users/adrien/Documents/SIMU/250107/
> SAVE DATA END
> SAVE METADATA
f_t 426500.0
ion_Vtkl01.038_VNW00.00_Vpiezo00.00_PhiNW000.00deg0303
/Users/adrien/Documents/SIMU/250107/
> SAVE DATA END
> SAVE METADATA
f_t 426000.0
ion_Vtkl01.038_VNW00.00_Vpiezo00.00_PhiNW000.00deg0304
/Users/adrien/Documents/SIMU/250107/
> SAVE DATA END
> SAVE METADATA
f_t 425500.0
ion_Vtkl01.038_VNW00.00_Vpiezo00.00_PhiNW000.00deg0305
/Users/adrien/Documents/SIMU/250107/
> SAVE DATA END
> SAVE METADATA
f_t 425000.0
ion_Vtkl01.038_VNW00.00_Vpiezo00.00_PhiNW000.00deg0306
/Users/adrien/Documents/SIMU/250107/
> SAVE DATA E